In [1]:
import pandas as pd
import os
import mlflow
import time
root_path = os.getcwd()
wind_farm_data = pd.read_csv(root_path+"/windfarm_data.csv",index_col=0)
wind_farm_data.head()

,temperature_00,wind_direction_00,wind_speed_00,temperature_08,wind_direction_08,wind_speed_08,temperature_16,wind_direction_16,wind_speed_16,power
2014-01-01,4.702022,106.742590,4.743292,7.189482,100.41638,6.593833,8.172301,99.28800,5.967206,1959.3535
2014-01-02,7.695733,98.036705,6.142716,9.977118,94.03181,4.383676,9.690135,204.25444,1.696528,1266.6239
2014-01-03,9.608235,274.061200,10.514304,10.840864,242.87563,16.869741,8.991079,250.26830,12.038399,7545.6797
2014-01-04,6.955563,257.910220,7.189170,5.317223,254.26170,9.069233,3.021174,284.06537,4.590843,3791.0408
2014-01-05,0.830547,265.394400,4.263086,2.480239,104.79496,3.042063,4.227131,263.41690,3.899182,880.6115


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("v6_hper_param").master("spark://spark-master:7077") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 03:50:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/04 03:50:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/04 03:50:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
mlflow_uri = "http://mlflow-server:8888"
mlflow.set_tracking_uri(mlflow_uri)
time.sleep(5)
#mlflow.set_experiment(mlflow_uri)
mlflow.set_experiment("v6_hper_param")

2025/05/04 03:50:53 INFO mlflow.tracking.fluent: Experiment with name 'v6_hper_param' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/325099155118866680', creation_time=1746330653025, experiment_id='325099155118866680', last_update_time=1746330653025, lifecycle_stage='active', name='v6_hper_param', tags={}>

In [4]:
def get_training_data(wind_farm_data):
    training_data = pd.DataFrame(wind_farm_data["2014-01-01":"2018-01-01"])
    X = training_data.drop(columns="power")
    y = training_data["power"]    
    return X,y

def get_validation_data(wind_farm_data):
    validation_data = pd.DataFrame(wind_farm_data["2018-01-01":"2019-01-01"])
    X = validation_data.drop(columns="power")
    y = validation_data["power"]
    return X,y


In [5]:
def get_weather_and_forecast():
    format_date = lambda pd_date : pd_date.strftime("%Y-%m-%d")
    today = pd.Timedelta('today').normalize()
    week_ago = today - pd.Timedelta(days=5)
    week_later = today + pd.Timedelta(days=5)


    past_power_output = pd.DataFrame(wind_farm_data[format_date(week_ago):format_date(today)])
    weather_and_forecast = pd.DataFrame(wind_farm_data[format_date(week_ago):format_date(week_later)])
    if len(weather_and_forecast) < 10 :
        past_power_output = pd.DataFrame(wind_farm_data).iloc[-10:-5]
        weather_and_forecast = pd.DataFrame(wind_farm_data).iloc[-10]
    return weather_and_forecast.drops(columns="power"),past_power_output["power"]

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


In [8]:
def train_keras_model(X,y):
    model = Sequential()
    model.add(Dense(100, input_shape = (X.shape[-1],), activation = "relu"))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    
    model.fit(X,y,epochs=30,batch_size=64, validation_split=0.2,verbose=1)
    return model

In [9]:
X_test, y_test = get_validation_data(wind_farm_data)
sample_x_test = X_test.iloc[:1]
print(sample_x_test.shape)
X_test.head()

(1, 9)


,temperature_00,wind_direction_00,wind_speed_00,temperature_08,wind_direction_08,wind_speed_08,temperature_16,wind_direction_16,wind_speed_16
2018-01-01,4.290002,99.383450,11.876492,5.606760,99.889145,11.322479,6.574509,106.049570,9.184392
2018-01-02,4.433410,101.933470,9.378234,6.396653,100.942276,9.721888,6.416043,107.416336,7.147915
2018-01-03,5.295909,111.764010,7.841654,6.982701,111.816930,7.587134,6.997799,109.317190,6.987251
2018-01-04,3.462663,101.464920,7.504305,5.784602,100.363434,6.056359,5.774397,103.493720,4.610690
2018-01-05,4.275198,102.928795,5.612769,6.063427,103.457120,5.794476,6.922954,101.492900,7.619769


In [10]:

import mlflow
import mlflow.tensorflow
import mlflow.keras
from mlflow.models.signature import infer_signature,set_signature


In [11]:
X_train, y_train = get_training_data(wind_farm_data)

with mlflow.start_run() as run:
    mlflow.tensorflow.autolog()

    model = train_keras_model(X_train,y_train)
    run_id = mlflow.active_run().info.run_id
    signature =  infer_signature(X_test, model.predict(X_test))
    model_uri = f"runs:/{run.info.run_id}/model"
    set_signature(model_uri, signature)

2025/05/04 03:52:30 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

/usr/local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `

Epoch 1/30
 1/19 ━━━━━━━━━━━━━━━━━━━━ 4s 244ms/step - loss: 10606917.0000

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10733498.0000 - val_loss: 7696635.0000
Epoch 2/30
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 13765286.0000

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10255010.0000 - val_loss: 7155647.5000
Epoch 3/30
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7678917.5000

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9247251.0000 - val_loss: 6541872.0000
Epoch 4/30
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 6424872.0000

...

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4645850.5000 - val_loss: 4222182.0000


2025/05/04 03:52:33 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: Cannot log input example or model signature for input with type <class 'pandas.core.frame.DataFrame'>. TensorFlow Keras autologging can only log input examples and model signatures for the following input types: numpy.ndarray, dict[string -> numpy.ndarray], tensorflow.keras.utils.Sequence, and tensorflow.data.Dataset (TensorFlow >= 2.1.0 required)
2025/05/04 03:52:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/05/04 03:52:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

🏃 View run righteous-smelt-997 at: http://mlflow-server:8888/#/experiments/325099155118866680/runs/cde3afed224745b78d337eaa7d70462d
🧪 View experiment at: http://mlflow-server:8888/#/experiments/325099155118866680


In [12]:

model_name = "power-forecasting-model"
print(run_id)


cde3afed224745b78d337eaa7d70462d


In [13]:
loaded_model = mlflow.tensorflow.load_model(f"runs:/{run_id}/model")
loaded_model.summary()

/usr/local/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │         1,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,204 (8.61 KB)

 Trainable params: 1,101 (4.30 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,103 (4.31 KB)

In [22]:
import mlflow
logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
y_pred = loaded_model.predict(pd.DataFrame(X_test))
y_pred

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step



/usr/local/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


,0
2018-01-01,1385.388672
2018-01-02,1326.935791
2018-01-03,1355.011108
2018-01-04,1196.310913
2018-01-05,1226.802856
...,...
2018-12-28,2987.428711
2018-12-29,2627.895264
2018-12-30,2779.671387
2018-12-31,1490.799438


In [23]:
import mlflow

artifact_path = "model"
model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run_id, artifact_path=artifact_path)
print("model_uri : ",model_uri)


model_version = mlflow.register_model(model_uri, model_name)

Successfully registered model 'power-forecasting-model'.
2024/10/31 15:33:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: power-forecasting-model, version 1


model_uri :  runs:/93dc9370ebde4904945ded68045947ef/model


Created version '1' of model 'power-forecasting-model'.
